## asdf

In [1]:
from twoplayer import make_pdf, construct_instance_from_dict, \
    store_instance_in_db, load_db, select_instance_dict_from_db, log

In [2]:
from sudokugen import instances, generate_puzzle, encodings, masks

In [3]:
from copy import deepcopy
import json
import random
import textwrap
from datetime import datetime

In [4]:
database = load_db("building-blocks-i.json")

db_variants = {
    1: ["1a"],
    2: ["2a"],
    3: ["3a", "3b"],
    4: ["4a", "4b", "4c"],
}

In [5]:
# # Either manually give a list of puzzle choices:
# puzzle_choices = [
#     {
#         ("A", 1): ("1a", 3),
#         ("A", 2): ("2a", 3),
#         ("A", 3): ("3a", 3),
#         ("B", 1): ("1a", 4),
#         ("B", 2): ("2a", 4),
#         ("B", 3): ("3a", 4),
#     },
#     {
#         ("A", 1): ("1a", 3),
#         ("A", 2): ("2a", 3),
#         ("A", 3): ("3a", 3),
#         ("B", 1): ("1a", 4),
#         ("B", 2): ("2a", 4),
#         ("B", 3): ("3a", 4),
#     },
# ]

In [6]:
# Or create a randomly generated list of puzzle choices,
# using a given level choice per player/puzzle_no
level_choice = {
    ("A", 1): 1,
    ("A", 2): 2,
    ("A", 3): 3,
    ("B", 1): 2,
    ("B", 2): 3,
    ("B", 3): 4,
}

def random_puzzle_choice(level_choice):
    puzzle_choice_dict = {}
    for player in ["A", "B"]:
        for puzzle_no in [1, 2, 3]:
            level = level_choice[(player, puzzle_no)]
            variant = random.choice(db_variants[level])
            puzzle_id = random.randint(1, len(database[variant]))
            puzzle_choice_dict[(player, puzzle_no)] = \
                (variant, puzzle_id)
    return puzzle_choice_dict

num_random_puzzles = 5

puzzle_choices = [
    random_puzzle_choice(level_choice)
    for _ in range(num_random_puzzles)
]

In [7]:
def select_puzzle(category, puzzle_id):
    return construct_instance_from_dict(
        select_instance_dict_from_db(
            database,
            category,
            puzzle_id
        )
    )

In [8]:
series_id = 1
volume_starting_id = 1

volume_id = volume_starting_id-1

for puzzle_choice in puzzle_choices:

    # Construct puzzle name and file name
    volume_id += 1
    puzzle_name = f"{series_id}-{volume_id}"
    filename = "puzzle-" + puzzle_name

    # Print what's happening..
    print(f"Creating puzzle {puzzle_name}..")
    print("- using this choice:")
    display(puzzle_choice)
    
    # Load puzzles
    instance_dict = {}
    for player in ["A", "B"]:
        for puzzle_no in [1, 2, 3]:
            category, puzzle_id = puzzle_choice[(player, puzzle_no)]
            instance_dict[(player, puzzle_no)] = select_puzzle(category, puzzle_id)
    
    # Construct meta info
    deco_list = [
        {
            "input color": "red",
            "input pattern": "north west",
            "output color": "blue",
            "output pattern": "horizontal",
        },
        {
            "input color": "blue",
            "input pattern": "horizontal",
            "output color": "red",
            "output pattern": "north west",
        },
    ]
    deco_index = random.randint(0, 1) # random choice of coloring for input-output
    print(deco_index)
    meta_info = {
        "level": {
            (player, puzzle_no): int(puzzle_choice[(player, puzzle_no)][0][0])
            for player in ["A", "B"]
            for puzzle_no in [1, 2, 3]
        },
        "deco": {
            "A": deco_list[deco_index],
            "B": deco_list[1-deco_index],
        },
        "subtitle": "(Pick one to solve)",
    }

    # Randomly construct a value permutation
    # that we will use for all instances
    value_list = list(range(1,10))
    random.shuffle(value_list)

    # Rearrange the instances appropriately
    arranged_instance_dict = {}

    for player in ["A", "B"]:
        for puzzle_no in [1, 2, 3]:

            instance = deepcopy(instance_dict[(player, puzzle_no)])

            # For player B, swap the input and output solutions and decoy values
            if player == "B":
                instance.swap_values(1, 2)
                instance.swap_values(3, 4)

            # Apply the value permutation
            instance.apply_value_permutation(value_list)

            # Shuffle the orientation of the puzzle
            instance.shuffle_orientation()

            arranged_instance_dict[(player, puzzle_no)] = instance

    # Make the actual PDF using LaTeX
    make_pdf(
        filename=filename,
        directory="output",
        instance_dict=arranged_instance_dict,
        meta_info=meta_info,
        puzzle_set_id=puzzle_name,
        two_page=True,
    )
    make_pdf(
        filename=filename+"-single",
        directory="output",
        instance_dict=arranged_instance_dict,
        meta_info=meta_info,
        puzzle_set_id=puzzle_name,
        two_page=False,
    )

    # Log what puzzles we used in the construction, and what value permutation
    logstr = datetime.now().strftime("=== %d-%m-%Y %H:%M:%S ===\n")
    logstr += f"- Created {filename}.pdf\n"
    logstr += f"- Puzzle set id: {puzzle_name}\n"
    for player in ["A", "B"]:
        for puzzle_no in [1, 2, 3]:
            category, puzzle_id = puzzle_choice[(player, puzzle_no)]
            logstr += f"- Puzzle #{puzzle_no} for {player}: {category}, {puzzle_id}\n"
    logstr += f"- Value permutation used: {value_list}\n"
    log(logstr)

Creating puzzle 1-1..
- using this choice:


{('A', 1): ('1a', 8),
 ('A', 2): ('2a', 14),
 ('A', 3): ('3a', 6),
 ('B', 1): ('2a', 15),
 ('B', 2): ('3b', 9),
 ('B', 3): ('4a', 6)}

1
Creating puzzle 1-2..
- using this choice:


{('A', 1): ('1a', 10),
 ('A', 2): ('2a', 24),
 ('A', 3): ('3b', 6),
 ('B', 1): ('2a', 20),
 ('B', 2): ('3a', 6),
 ('B', 3): ('4a', 3)}

0
Creating puzzle 1-3..
- using this choice:


...

0
Creating puzzle 1-4..
- using this choice:


{('A', 1): ('1a', 10),
 ('A', 2): ('2a', 30),
 ('A', 3): ('3b', 7),
 ('B', 1): ('2a', 29),
 ('B', 2): ('3a', 6),
 ('B', 3): ('4b', 1)}

1
Creating puzzle 1-5..
- using this choice:


{('A', 1): ('1a', 1),
 ('A', 2): ('2a', 30),
 ('A', 3): ('3b', 1),
 ('B', 1): ('2a', 13),
 ('B', 2): ('3a', 4),
 ('B', 3): ('4b', 1)}

1
